# Astrophysical Recast: Lab κ_R Bounds → Compact-Object QNM Constraints

This notebook propagates our laboratory null-result bounds on the curvature–EM coupling parameter κ_R to astrophysical regimes where they can constrain black hole quasi-normal mode (QNM) observables.

## Summary
- **Laboratory bounds** (from `null_results.tex`): κ_R < 5×10¹⁷ m² at B=10 T, R=10⁻²⁶ m⁻²
- **Astrophysical amplification**: Magnetar-like B~10⁸–10¹⁰ T, BH horizons R~(2M)⁻² ~ 10⁴–10¹⁰ m⁻²
- **Observable**: QNM frequency shift Δω/ω ∝ κ_R B² R (Karimabadi et al. scaling)
- **Target**: Compare to LIGO/Virgo ringdown sensitivity (~10⁻²–10⁻³ fractional precision)

## References
- **Laboratory null results**: `../papers/null_results.tex`
- **QNM theory**: Karimabadi et al. (arXiv:2508.13820)
- **Related analysis**: `../docs/related_papers/arxiv.2508.13820.md`

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

# Physical constants (SI)
c = 299792458.0  # m/s
G = 6.67430e-11  # m³/(kg·s²)
M_sun = 1.98847e30  # kg

# Configure plots
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

## 1. Laboratory Baseline

From `null_results.tex` (Section V, Eq. 47):
$$\kappa_R < 5 \times 10^{17}\,\mathrm{m}^2 \quad (B=10\,\mathrm{T},\, R=10^{-26}\,\mathrm{m}^{-2})$$

Effective coupling strength in lab:
$$S_{\mathrm{lab}} = \kappa_R B^2 R \sim (5\times10^{17})(100)(10^{-26}) = 5\times10^{-7}$$

In [ ]:
# Laboratory parameters
kappa_R_lab = 5e17  # m² (upper limit)
B_lab = 10.0  # T
R_lab = 1e-26  # m⁻²

S_lab = kappa_R_lab * B_lab**2 * R_lab
print(f"Laboratory effective coupling: S_lab = {S_lab:.2e}")
print(f"This is the dimensionless signal strength our experiment ruled out.")

## 2. Astrophysical Environments

### Black Hole Parameters
For a Schwarzschild BH of mass M:
$$r_s = 2GM/c^2, \quad R_{\mathrm{horizon}} \sim r_s^{-2} = \left(\frac{c^2}{2GM}\right)^2$$

### Magnetar Field Strengths
- Surface: B ~ 10⁸–10¹⁰ T (confirmed via cyclotron lines)
- Interior: B ~ 10¹⁰–10¹² T (theoretical estimates)

### Scaling Law
Assuming QNM shift scales as:
$$\frac{\Delta\omega}{\omega} \propto \kappa_R B^2 R$$

Then:
$$\frac{\Delta\omega}{\omega}_{\mathrm{astro}} = \left(\frac{\Delta\omega}{\omega}\right)_{\mathrm{lab}} \times \frac{B_{\mathrm{astro}}^2}{B_{\mathrm{lab}}^2} \times \frac{R_{\mathrm{astro}}}{R_{\mathrm{lab}}}$$

In [ ]:
# Define parameter grids
M_range = np.logspace(0, 2, 50) * M_sun  # 1–100 M_sun
B_range = np.logspace(8, 12, 50)  # 10⁸–10¹² T

# Compute curvature at horizon
def horizon_curvature(M):
    r_s = 2 * G * M / c**2
    return 1 / r_s**2

R_BH = horizon_curvature(M_range)

print(f"Mass range: {M_range[0]/M_sun:.1f}–{M_range[-1]/M_sun:.1f} M_sun")
print(f"Horizon curvature: {R_BH[0]:.2e}–{R_BH[-1]:.2e} m⁻²")
print(f"Field range: {B_range[0]:.2e}–{B_range[-1]:.2e} T")

## 3. Amplification Factor and Observable QNM Shift

Define amplification relative to lab:
$$A(M, B) = \frac{B^2}{B_{\mathrm{lab}}^2} \times \frac{R_{\mathrm{horizon}}(M)}{R_{\mathrm{lab}}}$$

Fractional QNM shift (assuming linear regime):
$$\left|\frac{\Delta\omega}{\omega}\right| = S_{\mathrm{lab}} \times A(M, B)$$

**Detectability criterion**: Compare to LIGO/Virgo ringdown precision:
- Optimistic: δω/ω ~ 10⁻³ (SNR > 50)
- Conservative: δω/ω ~ 10⁻² (SNR ~ 10)

In [ ]:
# Compute amplification map
M_grid, B_grid = np.meshgrid(M_range, B_range)
R_grid = horizon_curvature(M_grid)

A_grid = (B_grid / B_lab)**2 * (R_grid / R_lab)
Delta_omega_over_omega = S_lab * A_grid

# Detectability thresholds
threshold_optimistic = 1e-3
threshold_conservative = 1e-2

print(f"Amplification range: {A_grid.min():.2e}–{A_grid.max():.2e}")
print(f"QNM shift range: {Delta_omega_over_omega.min():.2e}–{Delta_omega_over_omega.max():.2e}")

## 4. Discovery Space Visualization

Plot showing parameter regions where:
1. Lab bounds already exclude detectable signals (Δω/ω < threshold)
2. Future astrophysical observations could probe remaining parameter space

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Left panel: Amplification factor
ax = axes[0]
contour = ax.contourf(M_grid/M_sun, B_grid, np.log10(A_grid), 
                      levels=20, cmap='viridis')
ax.contour(M_grid/M_sun, B_grid, np.log10(A_grid), 
           levels=10, colors='white', alpha=0.3, linewidths=0.5)
ax.set_xlabel('Black Hole Mass [M$_\odot$]')
ax.set_ylabel('Magnetic Field [T]')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_title('Amplification Factor A(M,B) [log₁₀]')
cbar = plt.colorbar(contour, ax=ax)
cbar.set_label('log₁₀(A)')

# Right panel: QNM shift with detection thresholds
ax = axes[1]
contour = ax.contourf(M_grid/M_sun, B_grid, np.log10(Delta_omega_over_omega), 
                      levels=20, cmap='plasma')
ax.contour(M_grid/M_sun, B_grid, Delta_omega_over_omega,
           levels=[threshold_optimistic], colors='cyan', 
           linewidths=2, linestyles='--')
ax.contour(M_grid/M_sun, B_grid, Delta_omega_over_omega,
           levels=[threshold_conservative], colors='white', 
           linewidths=2, linestyles='-')
ax.set_xlabel('Black Hole Mass [M$_\odot$]')
ax.set_ylabel('Magnetic Field [T]')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_title('Predicted QNM Shift |Δω/ω| [log₁₀]')
cbar = plt.colorbar(contour, ax=ax)
cbar.set_label('log₁₀(|Δω/ω|)')

# Add legend for contours
from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], color='cyan', linestyle='--', lw=2, 
           label='LIGO optimistic (10⁻³)'),
    Line2D([0], [0], color='white', linestyle='-', lw=2, 
           label='LIGO conservative (10⁻²)')
]
ax.legend(handles=legend_elements, loc='lower right')

plt.tight_layout()
plt.savefig('../results/analysis/qnm_recast_discovery_space.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nKey result:")
print(f"For B = 10¹⁰ T, M = 10 M_sun: Δω/ω ~ {Delta_omega_over_omega[np.argmin(np.abs(B_range - 1e10)), np.argmin(np.abs(M_range/M_sun - 10))]:.2e}")
print(f"This is {'ABOVE' if Delta_omega_over_omega[np.argmin(np.abs(B_range - 1e10)), np.argmin(np.abs(M_range/M_sun - 10))] > threshold_conservative else 'BELOW'} conservative LIGO threshold.")

## 5. Example Scenarios

### Scenario A: Stellar-Mass BH with Moderate Field
- M = 10 M_sun (typical LIGO source)
- B = 10⁸ T (weak magnetization)

### Scenario B: Magnetar-Strength BH
- M = 3 M_sun (low-mass BH)
- B = 10¹⁰ T (magnetar surface field)

### Scenario C: Hypermagnetic Compact Object
- M = 1.4 M_sun (neutron star mass)
- B = 10¹² T (magnetar interior)

In [ ]:
scenarios = [
    {"name": "A: Stellar BH + Weak Field", "M": 10*M_sun, "B": 1e8},
    {"name": "B: Low-Mass BH + Magnetar", "M": 3*M_sun, "B": 1e10},
    {"name": "C: NS Mass + Hypermagnetic", "M": 1.4*M_sun, "B": 1e12}
]

print("\n" + "="*80)
print("SCENARIO ANALYSIS")
print("="*80)

for s in scenarios:
    R_h = horizon_curvature(s["M"])
    A = (s["B"] / B_lab)**2 * (R_h / R_lab)
    shift = S_lab * A
    
    print(f"\n{s['name']}")
    print(f"  Mass: {s['M']/M_sun:.1f} M_sun")
    print(f"  Field: {s['B']:.2e} T")
    print(f"  Horizon curvature: {R_h:.2e} m⁻²")
    print(f"  Amplification: {A:.2e}")
    print(f"  |Δω/ω|: {shift:.2e}")
    
    if shift > threshold_optimistic:
        print(f"  → DETECTABLE by LIGO (optimistic)")
    elif shift > threshold_conservative:
        print(f"  → Marginally detectable (conservative LIGO)")
    else:
        print(f"  → Below LIGO threshold; lab bounds suffice")

## 6. Conservative Constraint Propagation

Invert the scaling to derive tighter κ_R bounds assuming astrophysical observations:

If future GW detections constrain |Δω/ω| < δ for a source with (M, B), then:
$$\kappa_R < \delta \times \frac{B_{\mathrm{lab}}^2}{B^2} \times \frac{R_{\mathrm{lab}}}{R_{\mathrm{horizon}}(M)} \times S_{\mathrm{lab}}^{-1}$$

In [ ]:
# Example: What κ_R bound would Scenario B imply if no signal at δ = 10⁻³?
M_test = 3 * M_sun
B_test = 1e10
delta_null = 1e-3

R_test = horizon_curvature(M_test)
kappa_R_implied = delta_null / (B_test**2 * R_test)

print(f"\nIf astrophysical null at M={M_test/M_sun:.1f} M_sun, B={B_test:.2e} T:")
print(f"  Current lab bound: κ_R < {kappa_R_lab:.2e} m²")
print(f"  Astro-implied bound: κ_R < {kappa_R_implied:.2e} m²")
print(f"  Improvement factor: {kappa_R_lab / kappa_R_implied:.2e}")

## 7. Conclusions

### Key Findings
1. **Laboratory bounds remain relevant**: For typical stellar-mass BHs with weak fields (B < 10⁸ T), predicted QNM shifts are below LIGO thresholds.

2. **Magnetar-BH systems are promising**: Objects with B ~ 10¹⁰ T and M ~ 1–10 M_sun produce Δω/ω ~ 10⁻³–10⁻¹, potentially detectable in next-generation GW observatories.

3. **Multi-scale EFT validation**: Combining lab nulls (high precision, weak fields) with astrophysical searches (strong fields, moderate precision) brackets κ_R from both sides.

### Next Steps
- Implement Karimabadi et al. WKB-6 QNM solver for precise frequency calculations
- Cross-check scaling assumptions against full GR + coupling perturbation theory
- Propose targeted GW follow-up for magnetar-BH merger candidates
- Update bounds when LIGO/Virgo/KAGRA ringdown catalogs expand

### Data Outputs
- Discovery space map: `../results/analysis/qnm_recast_discovery_space.png`
- Parameter tables exported for manuscript (see cell below)

In [ ]:
# Export summary table for manuscript
import pandas as pd

table_data = []
for s in scenarios:
    R_h = horizon_curvature(s["M"])
    A = (s["B"] / B_lab)**2 * (R_h / R_lab)
    shift = S_lab * A
    table_data.append({
        "Scenario": s["name"],
        "M [M_sun]": f"{s['M']/M_sun:.1f}",
        "B [T]": f"{s['B']:.1e}",
        "R [m^-2]": f"{R_h:.2e}",
        "Amplification": f"{A:.2e}",
        "|Δω/ω|": f"{shift:.2e}",
        "Detectable?": "Yes" if shift > threshold_optimistic else "Marginal" if shift > threshold_conservative else "No"
    })

df = pd.DataFrame(table_data)
print("\n" + df.to_string(index=False))
df.to_csv("../results/analysis/qnm_recast_scenarios.csv", index=False)
print("\nTable saved to: results/analysis/qnm_recast_scenarios.csv")